# LABORATORY 3
### RDD SOLUTION

In [1]:
registerRDD = sc.textFile("/data/students/bigdata_internet/lab3/register.csv")
stationsRDD = sc.textFile("/data/students/bigdata_internet/lab3/stations.csv")

In [2]:
# remove the header
headerRegister = registerRDD.first()
registerRDD = registerRDD.filter(lambda x: x!= headerRegister)
headerStations = stationsRDD.first()
stationsRDD = stationsRDD.filter(lambda x: x!= headerStations)

In [3]:
# split the data by \t
registerSplitRDD = registerRDD.map(lambda x: x.split("\t"))
stationsSplitRDD = stationsRDD.map(lambda x: x.split("\t"))

In [4]:
# filter wrong data (used_slots = 0 and free_slots = 0)
filterRegisterRDD = registerSplitRDD.filter\
(lambda l: int(l[2]) != 0 or int(l[3]) != 0)

#### 1.1.1 How many rows of data we obtain before and after the data cleaning above?

In [5]:
print("Before data cleaning: ",registerSplitRDD.count())
print("After data cleaning: ",filterRegisterRDD.count())

Before data cleaning:  25319028


[Stage 3:=================================================>         (5 + 1) / 6]

After data cleaning:  25104121


#### 2.1. Computes the criticality value

In [6]:
from datetime import datetime
# function to determine if a station is critical (free_slots = 0)
def isCritical(slots):
    if int(slots) == 0:
        return 1
    else:
        return 0

In [7]:
# extract from date: name of the day and hour
# exploit isCritical() function to give 0 or 1 for non critical or for critical 
# final format [(day, hour, id), (1, isCritical)] 
weekdayRegisterRDD = filterRegisterRDD.\
map(lambda date: ((datetime.strptime(date[1], "%Y-%m-%d %H:%M:%S")\
.strftime("%A"), datetime.strptime(date[1], "%Y-%m-%d %H:%M:%S").strftime("%H")\
, date[0]), [1, isCritical(date[3])]))
weekdayRegisterRDD.take(2)

[(('Thursday', '12', '1'), [1, 0]), (('Thursday', '12', '1'), [1, 0])]

In [8]:
# sum the 1 values and the number of criticals 
registerReadingsRDD = weekdayRegisterRDD.reduceByKey\
(lambda read1, read2: [read1[0] + read2[0], read1[1] + read2[1]])

In [9]:
# compute criticality
criticalityRDD = registerReadingsRDD.map(lambda l: (l[0], l[1][1]/l[1][0]))
criticalityRDD.take(2)

[(('Friday', '13', '160'), 0.0), (('Friday', '18', '160'), 0.0)]

#### 2.2. Selects only the critical pairs

In [10]:
threshold = 0.6
filterCriticalyRDD = criticalityRDD.filter(lambda crit: crit[1] > threshold)

In [11]:
print(filterCriticalyRDD.take(2))

[(('Monday', '01', '58'), 0.6239554317548747), (('Saturday', '00', '10'), 0.622107969151671)]


#### 2.3. Order the results by increasing criticality

In [12]:
# function to convert day name into number
def dayConversion(day):
    if day == "Monday":
        return 1
    elif day == "Tuesday":
        return 2
    elif day == "Wednesday":
        return 3
    elif day == "Thursday":
        return 4
    elif day == "Friday":
        return 5
    elif day == "Saturday":
        return 6
    elif day == "Sunday":
        return 7

In [13]:
# sort by criticality, id value, day of the week and hour
sortCritcalityRDD = filterCriticalyRDD\
.sortBy(lambda l: (l[1], l[0][2], dayConversion(l[0][0]), l[0][1]))

In [14]:
print(sortCritcalityRDD.take(5))

[(('Friday', '10', '9'), 0.6129032258064516), (('Saturday', '00', '10'), 0.622107969151671), (('Monday', '01', '58'), 0.6239554317548747), (('Friday', '22', '9'), 0.6258389261744967), (('Monday', '00', '58'), 0.6323119777158774)]


#### 2.4.  Store the sorted critical pairs in the output folder (also an argument of the application), by using a csv files (with header), where columns are separated by "tab"

In [60]:
# map to set id as key for register
sortedCriticalityRDD = sortCritcalityRDD\
.map(lambda l: (l[0][2], [l[0][0], l[0][1], l[1]]))

In [16]:
# map to set id as key for stations
stationsSplitRDD = stationsSplitRDD.map(lambda l: (l[0], [l[1], l[2]]))

In [17]:
joinStationsCriticalityRDD = stationsSplitRDD.join(sortedCriticalityRDD)

In [62]:
# insert tab to properly save the file
outputRDD = joinStationsCriticalityRDD\
.map(lambda x: \
f"{x[0]}\t{x[1][0][0]}\t{x[1][0][1]}\t{x[1][1][0]}\t{x[1][1][1]}\t{x[1][1][2]}")

In [20]:
outputRDD.saveAsTextFile("/user/s309709/LAB03/es1RDD")

#### 2.5. How many critical pairs do you obtain? Report also the complete output result of the applications.

In [21]:
print("Critical pairs obtained: ", outputRDD.count())

Critical pairs obtained:  5


In [22]:
print(outputRDD.take(5))

['10\t2.185206\t41.384875\tSaturday\t00\t0.622107969151671', '9\t2.185294\t41.385006\tFriday\t10\t0.6129032258064516', '9\t2.185294\t41.385006\tFriday\t22\t0.6258389261744967', '58\t2.170736\t41.377536\tMonday\t01\t0.6239554317548747', '58\t2.170736\t41.377536\tMonday\t00\t0.6323119777158774']


### SQL SOLUTION

In [65]:
dfRegister = spark.read.load("/data/students/bigdata_internet/lab3/register.csv",\
                             format = "csv",
                             sep = "\t",
                             header = True,
                             inferSchema = True)

In [66]:
dfStations = spark.read.load("/data/students/bigdata_internet/lab3/stations.csv",\
                             format = "csv",
                             sep = "\t",
                             header = True,
                             inferSchema = True)

#### 1.1.1 How many rows of data we obtain before and after the data cleaning above?

In [67]:
print("before data cleaning: ", dfRegister.count())

[Stage 128:==================================================>    (10 + 1) / 11]

before data cleaning:  25319028


In [68]:
dfRegisterFiltered = dfRegister.where("used_slots!=0 or free_slots!=0")

In [69]:
print("After data cleaning: ", dfRegisterFiltered.count())

[Stage 130:==================================================>    (10 + 1) / 11]

After data cleaning:  25104121


#### 2.1. Computes the criticality value

In [70]:
# function to determine if a station is critical (free_slots = 0)
def isCritical(slots):
    if int(slots) == 0:
        return 1
    else:
        return 0

spark.udf.register("isCritical", isCritical)

23/01/12 18:09:15 WARN analysis.SimpleFunctionRegistry: The function iscritical replaced a previously registered function.


<function __main__.isCritical(slots)>

In [71]:
# convert date in proper format and found critical stations
dfTimeslot = dfRegisterFiltered\
.selectExpr("station", "hour(timestamp) AS hour",\
           "date_format(timestamp, 'EEEE') AS day",\
           'isCritical(free_slots) AS critical')

In [72]:
dfTimeslot.show(3)

[Stage 132:>                                                        (0 + 1) / 1]

+-------+----+--------+--------+
|station|hour|     day|critical|
+-------+----+--------+--------+
|      1|  10|Thursday|       0|
|      1|  10|Thursday|       0|
|      1|  10|Thursday|       0|
+-------+----+--------+--------+
only showing top 3 rows



In [73]:
spark.udf.register("division", lambda x,y: float(x/y))

23/01/12 18:09:16 WARN analysis.SimpleFunctionRegistry: The function division replaced a previously registered function.


<function __main__.<lambda>(x, y)>

In [74]:
# evaluate criticality
dfCritcality = dfTimeslot.groupBy("station", "hour", "day")\
.agg({"station":"count", "critical":"sum"})\
.withColumnRenamed("count(station)", "total")\
.withColumnRenamed("sum(critical)", "critical")

In [75]:
dfCritcality = dfCritcality\
.selectExpr("station", "hour", "day", "division(critical,total) AS criticality")

In [76]:
dfCritcality.show(3)

[Stage 133:==================================================>    (10 + 1) / 11]

+-------+----+---------+-------------------+
|station|hour|      day|        criticality|
+-------+----+---------+-------------------+
|     54|  14|   Friday|0.07190635451505016|
|     55|   0|Wednesday|                0.0|
|     59|  16|   Friday|0.19630872483221476|
+-------+----+---------+-------------------+
only showing top 3 rows



In [77]:
# function to convert day name into number
def dayConversion(day):
    if day == "Monday":
        return 1
    elif day == "Tuesday":
        return 2
    elif day == "Wednesday":
        return 3
    elif day == "Thursday":
        return 4
    elif day == "Friday":
        return 5
    elif day == "Saturday":
        return 6
    elif day == "Sunday":
        return 7
    
spark.udf.register("dayConversion", dayConversion)

23/01/12 18:09:34 WARN analysis.SimpleFunctionRegistry: The function dayconversion replaced a previously registered function.


<function __main__.dayConversion(day)>

#### 2.2. Selects only the critical pairs

#### 2.3. Order the results by increasing criticality

In [78]:
dfCriticalFilter = dfCritcality.where("criticality >= 0.6")\
.selectExpr\
("station", "hour", "day", "criticality", "dayConversion(day) AS dayNum")\
.sort("criticality", "station", "dayNum", "hour")\
.select("station", "hour", "day", "criticality")   

In [79]:
dfCriticalFilter.show()

[Stage 136:============================>                       (111 + 13) / 200]

+-------+----+------+------------------+
|station|hour|   day|       criticality|
+-------+----+------+------------------+
|      9|   8|Friday|0.6129032258064516|
|     10|  22|Friday| 0.622107969151671|
|     58|  23|Sunday|0.6239554317548747|
|      9|  20|Friday|0.6258389261744967|
|     58|  22|Sunday|0.6323119777158774|
+-------+----+------+------------------+



#### 2.4. Store the sorted critical pairs in the output folder (also an argument of the application), by using a csv files (with header), where columns are separated by "tab"

In [80]:
dfJoin = dfStations\
.join(dfCriticalFilter, dfCriticalFilter.station == dfStations.id)

In [81]:
outputDF = dfJoin.select("id", "longitude", "latitude", "day", "hour", "criticality")

In [ ]:
outputDF.write.csv("/user/s309709/LAB03/es1_sql",sep= "\t",  header=True)

#### 2.5. How many critical pairs do you obtain? Report also the complete output result of the applications

In [83]:
print("Critical pairs obtained: ", outputDF.count())

[Stage 142:=========================================>          (161 + 13) / 200]

Critical pairs obtained:  5


In [84]:
outputDF.show()

[Stage 149:============================================>       (172 + 11) / 200]

+---+---------+---------+------+----+------------------+
| id|longitude| latitude|   day|hour|       criticality|
+---+---------+---------+------+----+------------------+
|  9| 2.185294|41.385006|Friday|   8|0.6129032258064516|
| 10| 2.185206|41.384875|Friday|  22| 0.622107969151671|
| 58| 2.170736|41.377536|Sunday|  23|0.6239554317548747|
|  9| 2.185294|41.385006|Friday|  20|0.6258389261744967|
| 58| 2.170736|41.377536|Sunday|  22|0.6323119777158774|
+---+---------+---------+------+----+------------------+

